In [1]:
def successor(digits):
    """
    Increment a list of ternary digits stored least significant digit first.
    Examples:
      [] -> [1]
      [2, 0, 1] -> [0, 1, 1]
      [2, 1, 0] -> [0, 2]
    """
    if digits is None:
        digits = []

    out = digits[:]  # do not mutate input
    i = 0
    carry = 1

    while carry:
        if i == len(out):          # extend if we ran out of digits
            out.append(0)
        s = out[i] + carry
        if s <= 2:                 # no overflow at this position
            out[i] = s
            carry = 0
        else:                      # 2 + 1 -> 0 with carry
            out[i] = 0
            i += 1                 # keep propagating

    # drop irrelevant trailing zeros
    while out and out[-1] == 0:
        out.pop()

    return out

In [2]:
assert successor([]) == [1]
assert successor([2, 0, 1]) == [0, 1, 1]
assert successor([2, 1, 0]) == [0, 2]     # [0, 2, 0] would also be acceptable
assert successor([2, 2, 2]) == [0, 0, 0, 1]

In [3]:
def _normalize(xs):
    if xs is None:
        return []
    ys = xs[:]  # do not mutate input
    while ys and ys[-1] == 0:  # drop trailing zeros
        ys.pop()
    return ys

def leq(A, B):
    """
    Return True if ternary list A <= ternary list B.
    Lists use least significant digit first, digits in {0,1,2}.
    Trailing zeros are irrelevant and ignored.
    """
    a, b = _normalize(A), _normalize(B)

    # First compare lengths (most significant position count)
    if len(a) != len(b):
        return len(a) < len(b)

    # Same length, compare from most significant digit down
    for i in range(len(a) - 1, -1, -1):
        if a[i] != b[i]:
            return a[i] < b[i]
    return True  # identical values

In [4]:
assert leq([2,0,1],[0,1,1]) is True
assert leq([2,0,1,0],[2,0,1]) is True
assert leq([0,1,1],[2,0,1]) is False

In [12]:
def tritwise_min(A, B):
    """
    Compute the element-wise (digit-wise) minimum of two ternary lists.
    Missing digits are implicitly zero.
    Example:
      tritwise_min([1,0,2], [1,1,1]) -> [1,0,1]
      tritwise_min([2,1],[0,2,1]) -> [0,1]
    """
    n = max(len(A), len(B))
    result = []

    for i in range(n):
        a = A[i] if i < len(A) else 0
        b = B[i] if i < len(B) else 0
        result.append(min(a, b))

    # Preserve an all-zero result; otherwise drop trailing zeros
    if any(d != 0 for d in result):
        while result and result[-1] == 0:
            result.pop()

    return result

In [11]:
assert tritwise_min([1,0,2], [1,1,1]) == [1,0,1]
assert tritwise_min([2,1], [0,2,1]) == [0,1]
assert tritwise_min([], [1,2]) == [0,0]

AssertionError: 

In [9]:
def tritwise_min(A, B):
    """
    Digit wise min on ternary lists (LSB first).
    Examples:
      tritwise_min([1,0,2],[1,1,1]) -> [1,0,1]
      tritwise_min([2,1],[0,2,1])   -> [0,1]
    """
    A = A or []
    B = B or []
    n = max(len(A), len(B))
    out = []

    for i in range(n):
        da = A[i] if i < len(A) else 0
        db = B[i] if i < len(B) else 0
        out.append(da if da <= db else db)

    while out and out[-1] == 0:
        out.pop()
    return out

In [13]:
assert tritwise_min([1,0,2],[1,1,1]) == [1,0,1]
assert tritwise_min([2,1],[0,2,1]) == [0,1]
assert tritwise_min([],[]) == []

In [21]:
def f(A, B):
    """
    Iterative tritwise MIN over all ternary lists from A to B inclusive.
    Assumes lists are LSB first. Requires leq(A, B) to be True.
    """
    if not leq(A, B):
        raise ValueError("A must be <= B")

    result = A or []
    curr = successor(A)

    while leq(curr, B):
        result = tritwise_min(result, curr)
        curr = successor(curr)

    return result

In [22]:
assert f([2,0,1],[1,1,1]) == [0,0,1]
assert f([],[]) == []
assert f([0,1,1],[0,1,1]) == [0,1,1]

In [23]:
def f_eff(A, B):
    """
    Tritwise MIN over all ternary values from A to B inclusive, in O(N).
    Lists are LSB-first; digits in {0,1,2}. Trailing zeros are irrelevant.
    """
    def digits_to_int(xs):
        if not xs:
            return 0
        value = 0
        p = 1
        for d in xs:
            value += d * p
            p *= 3
        return value

    # Normalize None to empty and compute scalar bounds
    A = A or []
    B = B or []

    a_val = digits_to_int(A)
    b_val = digits_to_int(B)
    if a_val > b_val:
        raise ValueError("A must be <= B")

    # Work up to the significant digit length of A/B
    def normalize(xs):
        ys = xs[:] if xs else []
        while ys and ys[-1] == 0:
            ys.pop()
        return ys

    n = max(len(normalize(A)), len(normalize(B)))
    out = []

    for i in range(n):
        block = 3 ** i         # length of each contiguous block for a fixed digit at position i
        period = block * 3     # full cycle length at digit i
        a_mod = a_val % period

        # Find smallest digit d in {0,1,2} such that [A,B] intersects the block where digit i == d
        chosen = 2  # default to highest; loop will try to lower it
        for d in (0, 1, 2):
            r = d * block
            # If A lies inside the block for digit d, we can take A itself
            if r <= a_mod <= r + block - 1:
                first = a_val
            else:
                delta = (r - a_mod) % period
                first = a_val + delta
            if first <= b_val:
                chosen = d
                break
        out.append(chosen)

    # Drop irrelevant trailing zeros
    while out and out[-1] == 0:
        out.pop()

    return out

# Basic parity checks with f
assert f_eff([2,0,1],[1,1,1]) == [0,0,1]
assert f_eff([],[]) == []
assert f_eff([0,1,1],[0,1,1]) == [0,1,1]



## Solution Notes and Explanation

### Representation
- **Ternary digits (trits)**: lists are LSB-first, digits in `{0,1,2}`.
- **Trailing zeros are irrelevant**: `[0,2]` and `[0,2,0,0]` represent the same value; helpers normalize by dropping trailing zeros.

### Functions
- `successor(digits)`: Adds 1 in base-3, LSB-first, without mutating input; drops trailing zeros in the result.
- `leq(A, B)`: Compares two ternary lists by value, ignoring trailing zeros.
- `tritwise_min(A, B)`: Digit-wise minimum with implicit zeros for missing positions; keeps an all-zero result intact, otherwise trims trailing zeros.
- `f(A, B)`: Iteratively computes the tritwise minimum across all values from `A` to `B` inclusive by walking the range with `successor` and folding with `tritwise_min`.
- `f_eff(A, B)`: Computes the same result in O(N) by reasoning per digit about whether the interval intersects the digit’s 0/1/2 blocks and picking the smallest present digit.

### Correctness intuition for `f_eff`
For a fixed position `i`, values modulo `3^(i+1)` form repeating blocks of length `3^i` where digit `i` equals `0`, `1`, or `2`. The interval `[A, B]` intersects some of these blocks. The minimal digit achievable at position `i` over the interval is the smallest `d ∈ {0,1,2}` such that `[A,B]` intersects the block where digit `i = d`. We detect the earliest value ≥ `A` landing in that block and check if it is ≤ `B`. Doing this for each position independently yields the digit-wise minimum over the whole interval.

### Edge cases handled
- `None` and empty lists are treated as zero.
- Different lengths are handled via implicit zeros.
- All-zero outcomes are preserved rather than collapsed to `[]`.
- Inputs with trailing zeros are normalized for comparisons.

### Complexity
- `f` (iterative): time Θ(K·N), where `K = |[A,B]|`; worst case `K = 3^N` ⇒ Θ(N·3^N). Space Θ(N).
- `f_eff` (per-digit): time Θ(N) with `N = max significant digits of A,B)`; space Θ(N). Integer arithmetic cost grows with `N`, but overall remains linear in the number of digits.

### Minimal examples
- `f([2,0,1],[1,1,1]) → [0,0,1]` and `f_eff` matches.
- Single-value intervals return that value (e.g., `[0,1,1]..[0,1,1] → [0,1,1]`).
